## Installation

In [1]:
!python -m spacy download en_core_web_md 

     |████████████████████████████████| 45.7 MB 205 kB/s  eta 0:00:01 MB 5.6 MB/s eta 0:00:08
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


## Imports

In [47]:
# get required imports
import pandas as pd
import numpy as np
import pickle as pkl
from collections import *
import Levenshtein
import spacy
import re
nlp = spacy.load("en_core_web_md")
from difflib import SequenceMatcher
import warnings
warnings. filterwarnings('ignore')

## Functions

In [51]:
def location_modifier(location):
    """(str) -> str
    makes string more readable
    
    Params:
       location (str) a part of object path that requires string comparison
    
    Returns:
        (str)
        A more readable and language friendly string that is easier to compare 
        against parsed test cases.
    """
    location = ' '.join(location.split('_'))
    all_list = (re.findall('([A-Z]+(?=[A-Z\n\s]|$))*([A-Z]?[a-z]*)', location))
    starter = ''
    for group in all_list:
        for string in group:
            if string == '':
                continue
            else:
                starter += string + ' ' 
    return starter.rstrip()

In [55]:
def get_sequence_score(string_a, string_b):
    """(str, str) -> int
    gives a cumulative longest common sequence score comparing words from one string to another
    
    Params:
       string_a (str) first string
       string_b (str) second string
       
    Returns:
        lcs (int) 
        Cumulativee longest common sequence score for words independent of order
    """
    list_a = string_a.split() #location
    list_b = string_b.split()
    lcs = 0
    for word_a in list_a:
        max_word_seq = -np.inf
        for word_b in list_b:
            seqMatch = SequenceMatcher(None, word_a, word_b)
            word_seq = seqMatch.find_longest_match(0, len(word_a), 0, len(word_b)).size
            if (word_seq > max_word_seq):
                max_word_seq = word_seq
        lcs += max_word_seq
    return lcs

In [56]:
def get_location(page, location):
    """(str, str) -> str
    finds the location most similar to the location in parsed test
    cases
    
    Params:
       page (str) page key for path_dict_c
       location (str) location property of parsed test case
       
    Returns:
        min_key (str) 
        key for path_dict_c[page] most similar to location
    """
    min_score = np.inf
    min_key = ''
    for key in path_dict_c[page].keys():
        lcs = get_sequence_score(location.lower(), location_modifier(key).lower())
        score = (len(location) - lcs)/len(location)
        #print(key, score, Levenshtein.distance(location.lower(), location_modifier(key).lower()), (len(location) - lcs)/lcs, lcs)
        if score < min_score:
            min_score = score
            min_key = key
        elif score == min_score:
            min_dist = Levenshtein.distance(location.lower(), location_modifier(min_key).lower())
            dist = Levenshtein.distance(location.lower(), location_modifier(key).lower())
            if dist < min_dist:
                min_score = score
                min_key = key
    return min_key

In [57]:
def get_alternate_location(location):
    """(str) -> str
    finds the location most similar to the location in parsed test
    cases
    
    Params:
       location (str) location property of parsed test case
       
    Returns:
        min_key (str) 
        key of alternate_dict_c that's most similar to location
    """
    min_score = np.inf
    min_key = ''
    for key in alternate_dict_c.keys():
        seqMatch = SequenceMatcher(None, location_modifier(key).lower(), location.lower())
        lcs = seqMatch.find_longest_match(0, len(location_modifier(key).lower()), 0, len(location.lower())).size
#         lcs = get_sequence_score(location.lower(), location_modifier(key).lower())
        #score = (len(location) - lcs)/len(location)
        score = (len(location_modifier(key).lower()) - lcs)/len(location_modifier(key).lower())
        #print(key, (len(location_modifier(key).lower()) - lcs)/len(location_modifier(key).lower()), Levenshtein.distance(location.lower(), key.lower()))
        if score < min_score:
            min_score = score
            min_key = key
        elif score == min_score:
            min_dist = Levenshtein.distance(location.lower(), location_modifier(min_key).lower())
            dist = Levenshtein.distance(location.lower(), location_modifier(key).lower())
            if dist < min_dist:
                min_score = score
                min_key = key
    return min_key

In [58]:
def page_str_modifier(page):
    """(str) -> str
    makes string more readable
    
    Params:
       page (str) a part of object path that requires string comparison
    
    Returns:
        (str)
        A more readable and language friendly string that is easier to compare 
        against parsed test cases.
    """
    if len(page.split('/')) > 1:
        page = ''.join(page.split('/')[1:])
        page = ''.join(page.split('_'))
    else:
        page = ''.join(page.split('_'))[1:]
    page = ' '.join(re.findall('[A-Z][a-z]+', page))
    return page

In [59]:
def get_page(page):
    """(str) -> str
    finds the page key most similar to the page value in parsed test
    cases
    
    Params:
       page (str) page property of parsed test case
       
    Returns:
        max_key (str) 
        key of path_dict_c that's most similar to page
    """
    max_sim = -np.inf
    max_key = ''
    for key in path_dict_c.keys():
        similarity = nlp(page).similarity(nlp(page_str_modifier(key)))
        if similarity > max_sim:
            max_sim = similarity
            max_key = key
    return max_key

In [62]:
def get_alternate_page(location, page):
    """(str, str) -> str
    finds the page key of alternate_dict_c[location] 
    most similar to the page value in parsed test cases
    
    Params:
       location (str) location key of alternate_dict_c
       page (str) page property of parsed test case
       
    Returns:
        max_key (str) 
        key of alternate_dict_c[location] that's most similar to page
    """
    max_sim = -np.inf
    max_key = ''
    for key in alternate_dict_c[location].keys():
        similarity = nlp(page.lower()).similarity(nlp(key.lower()))
        if similarity > max_sim:
            max_sim = similarity
            max_key = key
    return max_key

In [61]:
def get_object_path(parsed_test_case):
    """(dict) -> str or set
    finds the object path for a given parsed test case
    
    Params:
       parsed_test_case (dict) a parsed test case
       
    Returns:
        (str)
        The identified object path
        OR
        (set)
        A set of possible object path values
    """
    page = get_page(parsed_test_case['page'])
    if 'location' in parsed_test_case.keys():
        location = get_location(page, parsed_test_case['location'])
    else:
        location = get_location(page, parsed_test_case['value'])
    object_path_set = set()
    for element in path_dict_c[page][location]:
        if element in action_widget[parsed_test_case['V']]:
            object_path_set.add(element)        
    if len(object_path_set) == 1:
        return list(object_path_set)[0]
    elif len(object_path_set) > 1:
        return object_path_set
    elif len(object_path_set) == 0:
        if 'location' in parsed_test_case.keys():
            location = get_alternate_location(parsed_test_case['location'])
        else:
            location = get_alternate_location(parsed_test_case['value'])
        page = get_alternate_page(location, parsed_test_case['page'])
        for element in alternate_dict_c[location][page]:
            if element in action_widget[parsed_test_case['V']]:
                object_path_set.add(element)
        if len(object_path_set) == 1:
            return list(object_path_set)[0]
        elif len(object_path_set) > 1:
            return object_path_set
        elif len(object_path_set) == 0:
            object_path_set = set()
            page = get_page(parsed_test_case['page'])
            for element in path_dict_c[page].keys():
                for path in path_dict_c[page][element]:
                    if path in action_widget[parsed_test_case['V']]:
                        object_path_set.add(path)
            return object_path_set

## Data Structures

#### Dictionary containing all the functions and corresponding object paths

In [11]:
file_path = '/Users/anshulkaushal/desktop/funct-obj-dict.pkl'
func_obj_dict = pkl.load(open(file_path, "rb"))

#### The original path dictionary

In [12]:
path_dict = defaultdict(lambda: defaultdict(set))
for values in func_obj_dict.values():
    for value in values:
        get_path_list = value.split('/')
        path = '/'.join(get_path_list[0:-1])
        get_element_list = get_path_list[-1].split('_')
        if len(get_element_list) == 1:
            get_element_list = get_path_list[-1].split('-')
        element = get_element_list[1:]
        if len(get_element_list) > 1:
            element = '_'.join(get_element_list[1:])
            path_dict[path][element].add(value)
        else:
            element = get_element_list[0]
            path_dict[path][element].add(value)

#### Improved dictionary

In [13]:
imp_path_dict = path_dict.copy()
for key in path_dict['Module_Navigation'].keys():
    for x in path_dict['Module_Navigation'][key]:
        for page in imp_path_dict.keys():
            if key not in imp_path_dict[page].keys():
                imp_path_dict[page][key].add(x)

#### Much more improved dict

In [14]:
scripts_list = pkl.load(open('script_list.pkl', "rb"))

In [15]:
path_set = set()
for script in scripts_list:
    if isinstance(script, float):
        continue
    else:
        script_lines = script.split('\n')
        for idx, line in enumerate(script_lines):
            path_list = (re.findall("findTestObject\(\'(?:Object Repository\/)?([^\)\']*).*\)", line))
            for path in path_list:
                path_set.add(path)

In [16]:
path_dict_c = defaultdict(lambda: defaultdict(set))
for element in path_set:
    path_elements_list = element.split('/')
    if len(path_elements_list) == 2:
        ele_list = path_elements_list[-1].split('_')
        if len(ele_list) == 1:
            ele_list = ele_list[0].split('-')
            if len(ele_list) == 1:
                path_dict_c[path_elements_list[0]][ele_list[0]].add(element)
            else:
                ele = '_'.join(ele_list[1:])
                path_dict_c[path_elements_list[0]][ele].add(element)
        else:
            ele = '_'.join(ele_list[1:])
            path_dict_c[path_elements_list[0]][ele].add(element)
    else:
        path = path_elements_list[0]
        ele = ' '.join(path_elements_list[-1].split('_')[1:])
        path_dict_c[path][ele].add(element)

In [17]:
for key in path_dict_c['Module_Navigation'].keys():
    for x in path_dict_c['Module_Navigation'][key]:
        for page in path_dict_c.keys():
            if key not in path_dict_c[page].keys():
                path_dict_c[page][key].add(x)

#### Special case

In [18]:
rely_keys = set()
for key in path_dict_c['Page_Libraries'].keys():
    if key == 'relyOnName' or key == 'relyOnOrder':
        rely_keys.add(key)

In [19]:
for key in rely_keys:
    for element in path_dict_c['Page_Libraries'][key]:
        path_dict_c['Page_Libraries'][element.split('/')[-1]].add(element)
    del path_dict_c['Page_Libraries'][key]

#### Converting to file

In [76]:
with open('pickle_file_obj_path_dicts/path_dict_c.pkl', 'wb') as handle:
    pkl.dump(dict(path_dict_c), handle)

#### The dictionary for the alternate strategy

In [20]:
alternate_dict = defaultdict(lambda: defaultdict(set))
for key in imp_path_dict.keys():
    for values in imp_path_dict[key].values():
        for value in values:
            key_list = value.split('/')[-1].split('_')
            if len(key_list) == 1:
                key_list = key_list[0].split('-')
            if len(key_list) == 1:
                key_value = key_list[0]
            else:
                key_value = '_'.join(key_list[1:])
            page_list = value.split('/')[0:-1]
            page_value = '/'.join(page_list)
            alternate_dict[location_modifier(key_value).lower()][page_str_modifier(page_value)].add(value)

#### Improved alternate dictionary

In [24]:
alternate_dict_c = defaultdict(lambda: defaultdict(set))
for key in path_dict_c.keys():
    for values in path_dict_c[key].values():
        for value in values:
            key_list = value.split('/')[-1].split('_')
            if len(key_list) == 1:
                key_list = key_list[0].split('-')
            if len(key_list) == 1:
                key_value = key_list[0]
            else:
                key_value = '_'.join(key_list[1:])
            page_list = value.split('/')[0:-1]
            page_value = '/'.join(page_list)
            alternate_dict_c[location_modifier(key_value).lower()][page_str_modifier(page_value)].add(value)

#### Special case

In [25]:
for key in rely_keys:
    for element_key in alternate_dict_c[location_modifier(key).lower()].keys():
        for element in alternate_dict_c[location_modifier(key).lower()][element_key]:
            alternate_dict_c[location_modifier(element.split('/')[-1]).lower()][element_key].add(element)
    del alternate_dict_c[location_modifier(key).lower()]

#### Converting to file

In [79]:
with open('pickle_file_obj_path_dicts/alternate_dict_c.pkl', 'wb') as handle:
    pkl.dump(dict(alternate_dict_c), handle)

#### Action widget dictionary

In [26]:
action_widget = func_obj_dict.copy()
action_widget['enter'] = action_widget.pop('setText')
action_widget['input'] = action_widget['enter']
action_widget['wait'] = set()
for key in action_widget.keys():
    if 'wait' in key.lower():
        action_widget['wait'].update((action_widget[key]))
action_widget['get'] = action_widget.pop('getText')

#### Manual additions

In [27]:
action_widget['click'].add('Page_Libraries/Empty_Page/btn_createLibrary')
action_widget['click'].add('Page_Libraries/Library_Dialog/btn_Submit')
action_widget['click'].add('Page_Libraries/Question_Dialog/btn_Submit')
action_widget['wait'].add('Page_Libraries/Library_Dialog/txt_invalidMessage')
action_widget['get'].add('Page_Libraries/Library_Dialog/txt_invalidMessage')
action_widget['wait'].add('Page_Libraries/btn_addLibrary')
action_widget['click'].add('Page_Libraries/btn_addLibrary')
action_widget['input'].add('Page_Libraries/Library_Dialog/txt_nameField')
action_widget['get'].add('Page_Libraries/txt_SuccessfullMessage')

In [28]:
action_widget['verify'] = set()

In [29]:
for key in action_widget.keys():
    if 'verify' in key:
        action_widget['verify'].update(action_widget[key])

In [30]:
delete_keys = set()
for key in action_widget.keys():
    if 'verify' in key and key != 'verify':
        delete_keys.add(key)
    elif 'wait' in key and key != 'wait':
        delete_keys.add(key)
    else:
        continue

In [31]:
for key in delete_keys:
    del action_widget[key]

#### Converting to file 

In [80]:
with open('pickle_file_obj_path_dicts/action_widget.pkl', 'wb') as handle:
    pkl.dump(dict(action_widget), handle)

## Performance

In [63]:
test_cases = [[{'V': 'enter',
   'location': 'to email textbox',
   'value': 'admin1@mail.com',
   'page': 'login'},
  {'V': 'enter',
   'location': 'to password textbox',
   'value': 'Admin@123',
   'page': 'login'},
  {'V': 'click', 'value': 'Login', 'page': 'login'},
  {'V': 'wait',
   'value': 'title to be present for seconds',
   'time': 30,
   'page': 'login'}],
 [{'V': 'enter',
   'location': 'to email textbox',
   'value': 'invalid@wrong',
   'page': 'login'},
  {'V': 'enter',
   'location': 'to password textbox',
   'value': 'invalidpassword',
   'page': 'login'},
  {'V': 'click', 'value': 'Login', 'page': 'login'},
  {'V': 'wait',
   'value': 'email error message to appear for seconds',
   'time': 3,
   'page': 'login'},
  {'V': 'get', 'value': 'email error message', 'page': 'login'},
  {'V': 'check', 'page': 'login'},
  {'V': 'wait',
   'value': 'password error message to appear for seconds',
   'time': 3,
   'page': 'login'},
  {'V': 'get', 'value': 'password error message', 'page': 'login'},
  {'V': 'check', 'page': 'login'}],
 [{'V': 'click', 'value': 'the avatar', 'page': 'logout'},
  {'V': 'click', 'value': 'Sign out', 'page': 'logout'},
  {'V': 'wait', 'value': 'Login', 'time': 10, 'page': 'logout'}],
 [{'V': 'click', 'value': "user 's avatar", 'page': 'change password'},
  {'V': 'click', 'value': 'change password link', 'page': 'change password'},
  {'V': 'wait',
   'value': 'title to appear for s',
   'time': 10,
   'page': 'change password'},
  {'V': 'enter',
   'location': 'current password text box',
   'value': 'Admin@123',
   'page': 'change password'},
  {'V': 'enter',
   'location': 'new password text box',
   'value': 'Admin@1234',
   'page': 'change password'},
  {'V': 'enter',
   'location': 'to confirm password',
   'value': 'Admin@1234',
   'page': 'change password'},
  {'V': 'click', 'value': 'submit', 'page': 'change password'}],
 [{'V': 'click', 'value': 'the avatar', 'page': 'change password'},
  {'V': 'click', 'value': 'Change password', 'page': 'change password'},
  {'V': 'wait',
   'value': 'title to appear for s',
   'time': 10,
   'page': 'change password'},
  {'V': 'enter',
   'location': 'current password text box',
   'value': ' ',
   'page': 'change password'},
  {'V': 'enter',
   'location': 'new password text box',
   'value': ' ',
   'page': 'change password'},
  {'V': 'enter',
   'location': 'to confirm password',
   'value': ' ',
   'page': 'change password'},
  {'V': 'click', 'value': 'submit', 'page': 'change password'}],
 [{'V': 'click',
   'value': 'my profile link',
   'page': 'Update my personal info'},
  {'V': 'wait',
   'value': 'to appear for s',
   'time': 20,
   'page': 'Update my personal info'},
  {'V': 'enter',
   'location': 'full name text box',
   'value': 'new name',
   'page': 'Update my personal info'},
  {'V': 'enter',
   'location': 'phone number text box',
   'value': '1234567890',
   'page': 'Update my personal info'},
  {'V': 'click', 'value': 'submit', 'page': 'Update my personal info'}],
 [{'V': 'click', 'value': 'the avatar', 'page': 'Update my personal info'},
  {'V': 'click', 'value': 'My Profile', 'page': 'Update my personal info'},
  {'V': 'wait',
   'value': 'for email to appear for s',
   'time': 20,
   'page': 'Update my personal info'},
  {'V': 'enter',
   'location': 'full name text box',
   'value': ' ',
   'page': 'Update my personal info'},
  {'V': 'enter',
   'location': 'phone number text box',
   'value': '12345678',
   'page': 'Update my personal info'},
  {'V': 'click', 'value': 'submit', 'page': 'Update my personal info'}]]

In [64]:
steps_counter = 0
for test_case in test_cases:
    for step in test_case:
        if 'value' in step.keys():
            steps_counter += 1
            print(step)
            print(get_object_path(step))
            print()
        else:
            continue
    print('----------')
print(steps_counter)

{'V': 'enter', 'location': 'to email textbox', 'value': 'admin1@mail.com', 'page': 'login'}
Page_Login/tbx_Email

{'V': 'enter', 'location': 'to password textbox', 'value': 'Admin@123', 'page': 'login'}
Page_Login/tbx_Password

{'V': 'click', 'value': 'Login', 'page': 'login'}
Page_Login/btn_Login

{'V': 'wait', 'value': 'title to be present for seconds', 'time': 30, 'page': 'login'}
Page_ChangePassword/txt_Title

----------
{'V': 'enter', 'location': 'to email textbox', 'value': 'invalid@wrong', 'page': 'login'}
Page_Login/tbx_Email

{'V': 'enter', 'location': 'to password textbox', 'value': 'invalidpassword', 'page': 'login'}
Page_Login/tbx_Password

{'V': 'click', 'value': 'Login', 'page': 'login'}
Page_Login/btn_Login

{'V': 'wait', 'value': 'email error message to appear for seconds', 'time': 3, 'page': 'login'}
Page_Login/txt_EmailErrorMessage

{'V': 'get', 'value': 'email error message', 'page': 'login'}
Page_Login/txt_EmailErrorMessage

{'V': 'wait', 'value': 'password error me

In [65]:
test_cases = [[{'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'click', 'value': 'create library button', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'input',
   'location': 'to library name field textbox',
   'value': 'new library',
   'page': 'Create Libraries'},
  {'V': 'click', 'value': 'submit button', 'page': 'Create Libraries'}],
 [{'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'click', 'value': 'Create Library', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'set', 'page': 'Create Libraries'},
  {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
  {'V': 'wait',
   'value': 'for invalid message text to be present for s',
   'time': 20,
   'page': 'Create Libraries'},
  {'V': 'get', 'value': 'invalid message text', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'}],
 [{'V': 'wait',
   'value': 'Add Library',
   'time': 20,
   'page': 'Create Libraries'},
  {'V': 'click', 'value': 'Add Library', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'set', 'page': 'Create Libraries'},
  {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
  {'V': 'wait',
   'value': 'for success message text to be present for s',
   'time': 20,
   'page': 'Create Libraries'},
  {'V': 'get', 'value': 'success message text', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'}],
 [{'V': 'click', 'value': 'Add Library', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'},
  {'V': 'wait',
   'value': 'for invalid message text to be present for s',
   'time': 20,
   'page': 'Create Libraries'},
  {'V': 'get', 'value': 'invalid message text', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'click', 'value': 'Cancel', 'page': 'Create Libraries'}],
 [{'V': 'click',
   'value': 'library navigation item',
   'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'verify', 'page': 'Create Libraries'},
  {'V': 'click',
   'value': 'library navigation item',
   'page': 'Create Libraries'}],
 [{'V': 'click',
   'value': 'library name with rely on name',
   'page': 'Update Libraries'},
  {'V': 'click',
   'value': 'update option with rely on name',
   'page': 'Update Libraries'},
  {'V': 'verify', 'page': 'Update Libraries'},
  {'V': 'click', 'value': 'submit button', 'page': 'Update Libraries'}]]

In [66]:
steps_counter = 0
for test_case in test_cases:
    for step in test_case:
        if 'value' in step.keys():
            steps_counter += 1
            print(step)
            print(get_object_path(step))
            print()
        else:
            continue
    print('----------')
print(steps_counter)

{'V': 'click', 'value': 'create library button', 'page': 'Create Libraries'}
Page_Libraries/Empty_Page/btn_createLibrary

{'V': 'input', 'location': 'to library name field textbox', 'value': 'new library', 'page': 'Create Libraries'}
Page_Libraries/Library_Dialog/tbx_LibraryName

{'V': 'click', 'value': 'submit button', 'page': 'Create Libraries'}
{'Page_Libraries/Library_Dialog/btn_Submit', 'Page_Libraries/Question_Dialog/btn_Submit'}

----------
{'V': 'click', 'value': 'Create Library', 'page': 'Create Libraries'}
Page_Libraries/Empty_Page/btn_createLibrary

{'V': 'click', 'value': 'Submit', 'page': 'Create Libraries'}
{'Page_Libraries/Library_Dialog/btn_Submit', 'Page_Libraries/Question_Dialog/btn_Submit'}

{'V': 'wait', 'value': 'for invalid message text to be present for s', 'time': 20, 'page': 'Create Libraries'}
Page_Libraries/Library_Dialog/txt_invalidMessage

{'V': 'get', 'value': 'invalid message text', 'page': 'Create Libraries'}
Page_Libraries/Library_Dialog/txt_invalidMess